# locally generating results

In [1]:
from lib.my_initialization import *

# define values to run

In [2]:
import numpy as np
niter=100#250 #trials per worker
r_values=np.array([0.5])#,0.1,1.,1.5,2.,2.5,5.,10.])#cm
# r_values=np.array([0.1,0.5,1.,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.,2.5,5.,10.,20.])#cm
D_values=np.array([1.,2.,])#0.2,1.5,5.,10,20])#cm^2/s
A_values=np.array([20.25])#,25,50,100,156.25,250])[::-1]#cm^2
# A_values=np.array([20.25,25,39,50,56.25,100,156.25,189,250])[::-1]#cm^2
L_values=np.sqrt(A_values)#cm
# kappa_values=np.array([5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,250,500])#1/s
# beta_values=np.array([0.01,0.05,0.01,0.1,0.5,1])#cm
Dt_values=np.array([1e-7])
reflect_values=np.array([0])
set_second_values=np.array([0])
m_values=np.array([1.8772341309722325, 1.6375562704001745])
M_values=np.array([5.572315674840435,16.73559858353835])
num_trials_per_setting=1#6

Dratio_values=np.array([10])#,100,1000])
energy_gap_values=np.array(sorted([-10,10,0.,0.5,1,2,-1]))
temperature_energy_values=np.array([1.])

def comp_M_tilde(m):
    '''non-quadratic trend is guessed to make bounds'''
    val_lst=[]
    val_lst.append(np.pi**(m-1)/m)
    val_lst.append(np.pi**(m-1)/2)
    val_lst.append(np.pi/2)
    val_lst.append((np.pi/2)**(m-1))
    mn=np.min(val_lst)
    mx=np.max(val_lst)
    return mn,mx

def comp_kappa(M,m,r):
    '''returns a min/max estimate for kappa using the emergent universal law'''
    mn,mx=comp_M_tilde(m)
    kappa_mn=r**(-2*(m-1))*M/mx
    kappa_mx=r**(-2*(m-1))*M/mn
    return kappa_mn,kappa_mx

#iterate over settings, scheduling the longest jobs first
count=0
item_lst=[]
for set_second in set_second_values:
    for r in r_values:
        for D in D_values:
            for L in L_values:
                for Dt in Dt_values:
                    for reflect in reflect_values:
                        for m,M in zip(m_values,M_values):
                            # for kappa in kappa_values:
                            kappa_mn_mx=comp_kappa(M,m,r)
                            kappa_values=np.array(sorted(set([kappa_mn_mx[0],(kappa_mn_mx[0]+kappa_mn_mx[1])/2.,kappa_mn_mx[1]])))
                            kappa = np.mean(kappa_values)
                            for Dratio in Dratio_values:
                                for temperature_energy in temperature_energy_values:
                                    for energy_gap in energy_gap_values:
                                        num_trials=0
                                        while num_trials<num_trials_per_setting:
                                            num_trials+=1
                                            count=count+1
                                            Process=num_trials+1 #randomization seed
                                            item = (r,D,L,kappa,Dt,niter,Process,reflect,set_second,temperature_energy,energy_gap,Dratio)
                                            item_lst.append(item)
#                                             print(f"{r} {D} {L:.5f} {kappa:.5f} {Dt} {niter} {Process} {reflect} {set_second} {temperature_energy:.0f} {energy_gap:.0f} {Dratio:.0f}")
#                                             print(f"{r} {D} {L:.5f} {kappa:.5f} {Dt} {niter} {reflect} {set_second} {temperature_energy:.0f} {energy_gap:.0f} {Dratio:.0f}")
print(count)

28


# generate results

In [3]:
c_folder=f"{nb_dir}/../c/hybrid_modes"

In [4]:
def generate_results(item_lst,npartitions=9):
    '''filters Log and returns a merged pandas.Dataframe as *.csv
    returns the total runtime in years for all output files in folder.'''
#     input_fn_lst=get_files_in_folder(folder,trgt='.out.')
    print(f"generating results for {len(item_lst)} inputs...")
    def routine(item):
        jobid=1
        for f in item:
            if f>0:
                jobid*=(f+1)
        r,D,L,kappa,Dt,niter,Process,reflect,set_second,temperature_energy,energy_gap,Dratio=item
        os.chdir(c_folder)
        # os.system(f'./xrun.sh {r} {D} {L} {kappa} {Dt} {1} {Process} {reflect} {set_second} {temperature_energy} {energy_gap} {Dratio} > out.txt')
        os.system(f'./xrun.sh {r} {D} {L} {kappa} {Dt} {niter} {Process} {reflect} {set_second} {temperature_energy} {energy_gap} {Dratio} > Log/0.{jobid}.out.output')
        return True
    bag = db.from_sequence(sorted(item_lst), npartitions=9).map(routine)
    start = time.time()
    retval_lst = list(bag)
    print(f"the run time for generating results was {time.time()-start:.2f} seconds.")
    return retval_lst

In [5]:
retval_lst=generate_results(item_lst,npartitions=9)

generating results for 28 inputs...


Process ForkPoolWorker-9:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/timothytyree/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_byte

KeyboardInterrupt: 

In [ ]:
#local_logdown.sh
cd ../../../
mkdir python/data/local_results/Log
cp c/hybrid_modes/Log/ python/data/local_results/Log